In [1]:
import pandas as pd
import numpy as np

In [22]:
result = pd.read_csv('/content/drive/MyDrive/OCR_boaz_miniproj_2/OCR_boaz_miniproj_2/data/result_table', index_col = 0)

In [23]:
result

,outputs,label
0,불만과 불쾌감을 공개적으로 언급해 남북관계를 더욱 경색시키고 있,불만과 불쾌감을 공개적으로 언급해 남북관계를 더욱 경색시키고 있
1,대 의예과 1515학번 남학생 11명은 지난해 35월 학교 인근 그집집,대 의예과 1516학번 남학생 11명은 지난해 35월 학교 인근 고깃집
2,시 김종창 금감원장에게 청탁한 정황을 포착한 것으로 30일 확인됐,시 김종창 금감원장에게 청탁한 정황을 포착한 것으로 30일 확인됐
3,으로 알려지고 있다. 지난 5월 8일 구미시와 국가공단의 생명줄인 해,으로 알려지고 있다. 지난 5월 8일 구미시와 국가공단의 생명줄인 해
4,고의 스파클링 와인이다. 대표적인 스테미너 음식으로 꼽히는 장바구니,고의 스파클링 와인이다. 대표적인 스테미너 음식으로 꼽히는 장어구
...,...,...
840,"각 중단하고 박 원내대표에게 혐의가 있다면 당당히 기소하라 "" 며 ""","각 중단하고 박 원내대표에게 혐의가 있다면 당당히 기소하라""며 ""한"
841,등 번개가 치겠습니다. 비는 밤에 서쪽 지역부터 차차 그치겠습니.,둥 번개가 치겠습니다. 비는 밤에 서쪽 지역부터 차차 그치겠습니다.
842,골인하게 됐다. 예비 신랑은 1살 연상의 사업가 장모씨씨로 고대,골인하게 됐다. 예비 신랑은 1살 연상의 사업가 장모(32)씨로 고대
843,된다. 고무에 카본블랙주1을 섞은 일반타이어는 날씨가 추워질수록,된다. 고무에 카본블랙주1을 섞은 일반타이어는 날씨가 추워질수록


In [9]:
import numpy as np
from collections import Counter
from nltk import ngrams
import nltk.translate.bleu_score as bleu

In [10]:
# 토큰화 된 문장(tokens)에서 n-gram을 카운트
def simple_count(tokens, n):
  return Counter(ngrams(tokens, n))

In [11]:
candidate = "It is a guide to action which ensures that the military always obeys the commands of the party."
tokens = candidate.split() # 토큰화
result = simple_count(tokens, 1) # n = 1은 유니그램
print('유니그램 카운트 :',result)

유니그램 카운트 : Counter({('the',): 3, ('It',): 1, ('is',): 1, ('a',): 1, ('guide',): 1, ('to',): 1, ('action',): 1, ('which',): 1, ('ensures',): 1, ('that',): 1, ('military',): 1, ('always',): 1, ('obeys',): 1, ('commands',): 1, ('of',): 1, ('party.',): 1})


In [12]:
candidate = 'the the the the the the the'
tokens = candidate.split() # 토큰화
result = simple_count(tokens, 1) # n = 1은 유니그램
print('유니그램 카운트 :',result)

유니그램 카운트 : Counter({('the',): 7})


In [13]:
def count_clip(candidate, reference_list, n):
  # Ca 문장에서 n-gram 카운트
  ca_cnt = simple_count(candidate, n)
  max_ref_cnt_dict = dict()

  for ref in reference_list: 
    # Ref 문장에서 n-gram 카운트
    ref_cnt = simple_count(ref, n)

    # 각 Ref 문장에 대해서 비교하여 n-gram의 최대 등장 횟수를 계산.
    for n_gram in ref_cnt: 
      if n_gram in max_ref_cnt_dict:
        max_ref_cnt_dict[n_gram] = max(ref_cnt[n_gram], max_ref_cnt_dict[n_gram])
      else:
        max_ref_cnt_dict[n_gram] = ref_cnt[n_gram]

  return {
        # count_clip = min(count, max_ref_count)
        n_gram: min(ca_cnt.get(n_gram, 0), max_ref_cnt_dict.get(n_gram, 0)) for n_gram in ca_cnt
     }

In [14]:
candidate = 'the the the the the the the'
references = [
    'the cat is on the mat',
    'there is a cat on the mat'
]
result = count_clip(candidate.split(),list(map(lambda ref: ref.split(), references)),1)
print('보정된 유니그램 카운트 :',result)

보정된 유니그램 카운트 : {('the',): 2}


In [15]:
def modified_precision(candidate, reference_list, n):
  clip_cnt = count_clip(candidate, reference_list, n) 
  total_clip_cnt = sum(clip_cnt.values()) # 분자

  cnt = simple_count(candidate, n)
  total_cnt = sum(cnt.values()) # 분모

  # 분모가 0이 되는 것을 방지
  if total_cnt == 0: 
    total_cnt = 1

  # 분자 : count_clip의 합, 분모 : 단순 count의 합 ==> 보정된 정밀도
  return (total_clip_cnt / total_cnt)

In [16]:
result = modified_precision(candidate.split(), list(map(lambda ref: ref.split(), references)), n=1)
print('보정된 유니그램 정밀도 :',result)

보정된 유니그램 정밀도 : 0.2857142857142857


In [17]:
# Ca 길이와 가장 근접한 Ref의 길이를 리턴하는 함수
def closest_ref_length(candidate, reference_list):
  ca_len = len(candidate) # ca 길이
  ref_lens = (len(ref) for ref in reference_list) # Ref들의 길이
  # 길이 차이를 최소화하는 Ref를 찾아서 Ref의 길이를 리턴
  closest_ref_len = min(ref_lens, key=lambda ref_len: (abs(ref_len - ca_len), ref_len))
  return closest_ref_len

In [18]:
def brevity_penalty(candidate, reference_list):
  ca_len = len(candidate)
  ref_len = closest_ref_length(candidate, reference_list)

  if ca_len > ref_len:
    return 1

  # candidate가 비어있다면 BP = 0 → BLEU = 0.0
  elif ca_len == 0 :
    return 0
  else:
    return np.exp(1 - ref_len/ca_len)

In [19]:
def bleu_score(candidate, reference_list, weights=[0.25, 0.25, 0.25, 0.25]):
  bp = brevity_penalty(candidate, reference_list) # 브레버티 패널티, BP

  p_n = [modified_precision(candidate, reference_list, n=n) for n, _ in enumerate(weights,start=1)] 
  # p1, p2, p3, ..., pn
  score = np.sum([w_i * np.log(p_i) if p_i != 0 else 0 for w_i, p_i in zip(weights, p_n)])
  return bp * np.exp(score)

In [20]:
candidate = 'It is a guide to action which ensures that the military always obeys the commands of the party'
references = [
    'It is a guide to action that ensures that the military will forever heed Party commands',
    'It is the guiding principle which guarantees the military forces always being under the command of the Party',
    'It is the practical guide for the army always to heed the directions of the party'
]

print('실습 코드의 BLEU :',bleu_score(candidate.split(),list(map(lambda ref: ref.split(), references))))
print('패키지 NLTK의 BLEU :',bleu.sentence_bleu(list(map(lambda ref: ref.split(), references)),candidate.split()))

실습 코드의 BLEU : 0.5045666840058485
패키지 NLTK의 BLEU : 0.5045666840058485


In [24]:
result

,outputs,label
0,불만과 불쾌감을 공개적으로 언급해 남북관계를 더욱 경색시키고 있,불만과 불쾌감을 공개적으로 언급해 남북관계를 더욱 경색시키고 있
1,대 의예과 1515학번 남학생 11명은 지난해 35월 학교 인근 그집집,대 의예과 1516학번 남학생 11명은 지난해 35월 학교 인근 고깃집
2,시 김종창 금감원장에게 청탁한 정황을 포착한 것으로 30일 확인됐,시 김종창 금감원장에게 청탁한 정황을 포착한 것으로 30일 확인됐
3,으로 알려지고 있다. 지난 5월 8일 구미시와 국가공단의 생명줄인 해,으로 알려지고 있다. 지난 5월 8일 구미시와 국가공단의 생명줄인 해
4,고의 스파클링 와인이다. 대표적인 스테미너 음식으로 꼽히는 장바구니,고의 스파클링 와인이다. 대표적인 스테미너 음식으로 꼽히는 장어구
...,...,...
840,"각 중단하고 박 원내대표에게 혐의가 있다면 당당히 기소하라 "" 며 ""","각 중단하고 박 원내대표에게 혐의가 있다면 당당히 기소하라""며 ""한"
841,등 번개가 치겠습니다. 비는 밤에 서쪽 지역부터 차차 그치겠습니.,둥 번개가 치겠습니다. 비는 밤에 서쪽 지역부터 차차 그치겠습니다.
842,골인하게 됐다. 예비 신랑은 1살 연상의 사업가 장모씨씨로 고대,골인하게 됐다. 예비 신랑은 1살 연상의 사업가 장모(32)씨로 고대
843,된다. 고무에 카본블랙주1을 섞은 일반타이어는 날씨가 추워질수록,된다. 고무에 카본블랙주1을 섞은 일반타이어는 날씨가 추워질수록


In [26]:
result.loc[0]['outputs']

'불만과 불쾌감을 공개적으로 언급해 남북관계를 더욱 경색시키고 있'

In [31]:
candidate = result.loc[0]['outputs']
references = [result.loc[0]['label']]

print(candidate)

print(references)

print(bleu.sentence_bleu(list(map(lambda ref: ref.split(), references)),candidate.split()))

불만과 불쾌감을 공개적으로 언급해 남북관계를 더욱 경색시키고 있
['불만과 불쾌감을 공개적으로 언급해 남북관계를 더욱 경색시키고 있']
1.0


In [37]:
def evaluation_bleu(df):
  data = df.copy()
  data['bleu_score'] = None
  for idx in data.index:
    candidate = data.loc[idx]['outputs']
    references = [data.loc[idx]['label']]

    data.loc[idx]['bleu_score'] = bleu.sentence_bleu(list(map(lambda ref: ref.split(), references)),candidate.split())


  print('test data 평균 성능:',np.mean(data['bleu_score']))
  return data

In [38]:
eval = evaluation_bleu(result)

/usr/local/lib/python3.10/dist-packages/nltk/translate/bleu_score.py:552: UserWarning: 
The hypothesis contains 0 counts of 3-gram overlaps.
Therefore the BLEU score evaluates to 0, independently of
how many N-gram overlaps of lower order it contains.
Consider using lower n-gram order or use SmoothingFunction()
  warnings.warn(_msg)
/usr/local/lib/python3.10/dist-packages/nltk/translate/bleu_score.py:552: UserWarning: 
The hypothesis contains 0 counts of 4-gram overlaps.
Therefore the BLEU score evaluates to 0, independently of
how many N-gram overlaps of lower order it contains.
Consider using lower n-gram order or use SmoothingFunction()
  warnings.warn(_msg)
/usr/local/lib/python3.10/dist-packages/nltk/translate/bleu_score.py:552: UserWarning: 
The hypothesis contains 0 counts of 2-gram overlaps.
Therefore the BLEU score evaluates to 0, independently of
how many N-gram overlaps of lower order it contains.
Consider using lower n-gram order or use SmoothingFunction()
  warnings.warn(_

test data 평균 성능: 0.7472296744677713


In [39]:
eval

,outputs,label,bleu_score
0,불만과 불쾌감을 공개적으로 언급해 남북관계를 더욱 경색시키고 있,불만과 불쾌감을 공개적으로 언급해 남북관계를 더욱 경색시키고 있,1.0
1,대 의예과 1515학번 남학생 11명은 지난해 35월 학교 인근 그집집,대 의예과 1516학번 남학생 11명은 지난해 35월 학교 인근 고깃집,0.581431
2,시 김종창 금감원장에게 청탁한 정황을 포착한 것으로 30일 확인됐,시 김종창 금감원장에게 청탁한 정황을 포착한 것으로 30일 확인됐,1.0
3,으로 알려지고 있다. 지난 5월 8일 구미시와 국가공단의 생명줄인 해,으로 알려지고 있다. 지난 5월 8일 구미시와 국가공단의 생명줄인 해,1.0
4,고의 스파클링 와인이다. 대표적인 스테미너 음식으로 꼽히는 장바구니,고의 스파클링 와인이다. 대표적인 스테미너 음식으로 꼽히는 장어구,0.840896
...,...,...,...
840,"각 중단하고 박 원내대표에게 혐의가 있다면 당당히 기소하라 "" 며 ""","각 중단하고 박 원내대표에게 혐의가 있다면 당당히 기소하라""며 ""한",0.570675
841,등 번개가 치겠습니다. 비는 밤에 서쪽 지역부터 차차 그치겠습니.,둥 번개가 치겠습니다. 비는 밤에 서쪽 지역부터 차차 그치겠습니다.,0.72598
842,골인하게 됐다. 예비 신랑은 1살 연상의 사업가 장모씨씨로 고대,골인하게 됐다. 예비 신랑은 1살 연상의 사업가 장모(32)씨로 고대,0.750624
843,된다. 고무에 카본블랙주1을 섞은 일반타이어는 날씨가 추워질수록,된다. 고무에 카본블랙주1을 섞은 일반타이어는 날씨가 추워질수록,1.0


In [41]:
eval.to_csv('/content/drive/MyDrive/OCR_boaz_miniproj_2/OCR_boaz_miniproj_2/data/정을_trocr_test_bleu(0.75).csv')